In [4]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib import rc
plt.rc('font', family='serif')
import tensorflow as tf
import time

In [ ]:
def scrape_folder(folder, num_iters, metric_names = ['psnr', 'ssim', 'lpips']):
  stats = {}
  for i_metric, metric_name in enumerate(metric_names):
    filename = os.path.join(folder, 'test_preds', f'metric_{metric_name}_{num_iters}.txt')
    with open(filename, mode='r') as f:
      v = np.array([float(s) for s in f.readline().split(' ')])
    stats[metric_name] = np.mean(v)

  tic = time.time()
  grab_tags = ['train_steps_per_sec', 'num_params', 'train_num_params']
  grabbed_tags = {k:[] for k in grab_tags}
  for pattern in ['events*']:#, 'eval/events*']:
    for event_file in glob.glob(os.path.join(folder, pattern)):
      for event in tf.compat.v1.train.summary_iterator(event_file):
        value = event.summary.value
        if len(value) > 0:
          tag = event.summary.value[0].tag
          if tag in grab_tags:
            grabbed_tags[tag].append(np.array(tf.make_ndarray(event.summary.value[0].tensor)).item())

  if grabbed_tags['train_steps_per_sec']:
    steps_per_sec = np.percentile(np.array(grabbed_tags['train_steps_per_sec']), 95)
    stats['num_hours'] = (num_iters / steps_per_sec) / (60**2)
  else:
    stats['num_hours'] = np.nan

  if grabbed_tags['num_params']:
    stats['mega_params'] = int(np.max(grabbed_tags['num_params'])) / (1000000) # in millions
  elif grabbed_tags['train_num_params']:
    stats['mega_params'] = int(np.max(grabbed_tags['train_num_params'])) / (1000000) # in millions
  else:
    stats['mega_params'] = np.nan

  return stats

def render_table(names, data, precisions, rank_order, suffixes=None, hlines = []):
  def rankify(x, order):
    assert len(x.shape) == 1
    if order == 0:
      return np.full_like(x, 1e5, dtype=np.int32)
    u = np.sort(np.unique(x))
    if order == 1:
      u = u[::-1]
    r = np.zeros_like(x, dtype=np.int32)
    for ui, uu in enumerate(u):
      mask = x == uu
      r[mask] = ui
    return np.int32(r)

  tags = ['   \cellcolor{red}',
          '\cellcolor{orange}',
          '\cellcolor{yellow}',
          '                  ']

  max_len = max([len(v) for v in list(names)])
  names_padded = [v + ' '*(max_len-len(v)) for v in names]

  data_quant = np.round((data * 10.**(np.array(precisions)[None, :]))) / 10.**(np.array(precisions)[None, :])
  if suffixes is None:
    suffixes = [''] * len(precisions)

  tagranks = []
  for d in range(data_quant.shape[1]):
    tagranks.append(np.clip(rankify(data_quant[:,d], rank_order[d]), 0, len(tags)-1))
  tagranks = np.stack(tagranks, -1)

  for i_row in range(len(names)):
    line = ''
    if i_row in hlines:
      line += '\\hline\n'
    line += names_padded[i_row]
    for d in range(data_quant.shape[1]):
      line += ' & '
      if rank_order[d] != 0 and not np.isnan(data[i_row,d]):
        line += tags[tagranks[i_row, d]]
      if np.isnan(data[i_row,d]):
        line += ' - '
      else:
        assert precisions[d] >= 0
        line += ('{:' + f'0.{precisions[d]}f' + '}').format(data_quant[i_row,d]) + suffixes[d]
    line += ' \\\\'
    print(line)


Reproducing Ref-NeRF's Results

In [ ]:
root_folder = '/cns/lu-d/home/buff/dorverbin/nerf/mipnerf360/'
scene_names = ['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship']
metric_names = ['psnr', 'ssim', 'lpips', 'normals_mae']

models_meta = {} # folder : latex_name
models_meta[None] = 'ref-NeRF (from the paper)'
models_meta['blender_refnerf_453115176'] = 'ref-NeRF 360'

NUM_ITERS = 250000

all_stats = []
avg_stats = []
for model_path in models_meta.keys():

  if model_path is None:
    # Inject the numbers from the paper.
    psnrs = [35.83, 25.79, 33.91, 37.72, 36.25, 35.41, 36.76, 30.28]
    ssims = [0.984, 0.937, 0.983, 0.984, 0.981, 0.983, 0.992, 0.880]
    lpips = [0.017, 0.059, 0.019, 0.022, 0.018, 0.022, 0.007, 0.139]
    # There was a bug in MAE computation for ['ficus', 'lego', 'mic'] in the paper, those numbers shouldn't be trusted.
    nmaes = [19.852, 27.853, np.nan, 13.211, np.nan, 9.531, np.nan, 31.707]
    train_times = [np.nan]*len(psnrs)
    model_sizes = [np.nan]*len(psnrs)
    scene_stats = []
    for p, s, l, n, tt, ms in zip(psnrs, ssims, lpips, nmaes, train_times, model_sizes):
      scene_stats.append({'psnr': p, 'ssim': s, 'lpips': l, 'normals_mae': n, 'num_hours': tt, 'mega_params': ms})
    avg_stats.append({k: type(scene_stats[0][k])(np.mean([s[k] for s in scene_stats])) for k in scene_stats[0].keys()})
    all_stats.append(scene_stats)
    continue;

  scene_stats = []
  for scene_name in scene_names:
    folder = os.path.join(root_folder, model_path, scene_name)
    stats = scrape_folder(folder, NUM_ITERS, metric_names=metric_names)
    print(model_path, scene_name, stats)
    scene_stats.append(stats)
  avg_stats.append({k: type(scene_stats[0][k])(np.mean([s[k] for s in scene_stats])) for k in scene_stats[0].keys()})
  all_stats.append(scene_stats)
  print(model_path, avg_stats[-1])

blender_refnerf_453115176 chair {'psnr': 35.43609601974487, 'ssim': 0.9821090731024742, 'lpips': 0.019048675729427488, 'normals_mae': 20.85330159, 'num_hours': 6.571275680175564, 'mega_params': 0.71323}
blender_refnerf_453115176 drums {'psnr': 25.852998418807985, 'ssim': 0.9371991902589798, 'lpips': 0.06024521112442017, 'normals_mae': 27.641556060000003, 'num_hours': 6.567777006412732, 'mega_params': 0.71323}
blender_refnerf_453115176 ficus {'psnr': 30.604970512390135, 'ssim': 0.966888021826744, 'lpips': 0.03917563056573272, 'normals_mae': 40.80395823, 'num_hours': 6.5770709353398535, 'mega_params': 0.71323}
blender_refnerf_453115176 hotdog {'psnr': 37.485798597335815, 'ssim': 0.9824606701731682, 'lpips': 0.02336025163065642, 'normals_mae': 8.700470121, 'num_hours': 6.570256454244849, 'mega_params': 0.71323}
blender_refnerf_453115176 lego {'psnr': 36.05616488456726, 'ssim': 0.9799841883778572, 'lpips': 0.01861008478794247, 'normals_mae': 24.550344864999996, 'num_hours': 6.5775860225172

In [ ]:
names = list(models_meta.values())
data = np.stack([list(s.values()) for s in avg_stats])
precisions = [2, 3, 3, 2, 2, 1]
rank_order = [1, 1, -1, -1, 0, 0]  # +1 = higher is better, -1 = lower is better, 0 = do not color code
suffixes = ['', '', '', '', '', 'M']
render_table(names, data, precisions, rank_order, suffixes=suffixes)

ref-NeRF (from the paper) &    \cellcolor{red}33.99 &    \cellcolor{red}0.966 &    \cellcolor{red}0.038 &  -  &  -  &  -  \\
ref-NeRF 360              & \cellcolor{orange}33.46 & \cellcolor{orange}0.963 & \cellcolor{orange}0.040 &    \cellcolor{red}23.38 & 6.57 & 0.7M \\


In [ ]:
names = list(models_meta.values())

precisions = [2, 3, 3, 2]
rank_orders = [1, 1, -1, -1]

name_map = {s: s for s in scene_names}
pretty_scene_names = [name_map[s] for s in scene_names]

for i_metric, metric in enumerate(['psnr', 'ssim', 'lpips', 'normals_mae']):
  print(metric)
  precision = precisions[i_metric]
  rank_order = rank_orders[i_metric]

  print(' & ' + ' & '.join(['\\textit{' + s + '}' for s in pretty_scene_names]) + ' \\\\\\hline')
  data = np.array([np.array([s[metric] for s in scene_stats]) for scene_stats in all_stats])
  render_table(names, data, [precision] * len(scene_names), [rank_order] * len(scene_names))
  print()

psnr
 & \textit{chair} & \textit{drums} & \textit{ficus} & \textit{hotdog} & \textit{lego} & \textit{materials} & \textit{mic} & \textit{ship} \\\hline
ref-NeRF (from the paper) &    \cellcolor{red}35.83 & \cellcolor{orange}25.79 &    \cellcolor{red}33.91 &    \cellcolor{red}37.72 &    \cellcolor{red}36.25 &    \cellcolor{red}35.41 &    \cellcolor{red}36.76 & \cellcolor{orange}30.28 \\
ref-NeRF 360              & \cellcolor{orange}35.44 &    \cellcolor{red}25.85 & \cellcolor{orange}30.60 & \cellcolor{orange}37.49 & \cellcolor{orange}36.06 & \cellcolor{orange}35.04 & \cellcolor{orange}36.73 &    \cellcolor{red}30.51 \\

ssim
 & \textit{chair} & \textit{drums} & \textit{ficus} & \textit{hotdog} & \textit{lego} & \textit{materials} & \textit{mic} & \textit{ship} \\\hline
ref-NeRF (from the paper) &    \cellcolor{red}0.984 &    \cellcolor{red}0.937 &    \cellcolor{red}0.983 &    \cellcolor{red}0.984 &    \cellcolor{red}0.981 &    \cellcolor{red}0.983 &    \cellcolor{red}0.992 & \cellcolor{

In [ ]:
root_folder = '/cns/lu-d/home/buff/dorverbin/nerf/mipnerf360/'
scene_names = ['coffee', 'helmet3', 'musclecar', 'phong_envmap', 'teapot2', 'toaster']
metric_names = ['psnr', 'ssim', 'lpips', 'normals_mae']

models_meta = {} # folder : latex_name
models_meta[None] = 'ref-NeRF (from the paper)'
models_meta['blender_refnerf_453115176'] = 'ref-NeRF 360'

NUM_ITERS = 250000

all_stats = []
avg_stats = []
for model_path in models_meta.keys():

  if model_path is None:
    # Inject the numbers from the paper.
    psnrs = [34.21, 29.68, 30.82, 47.46, 47.90, 25.70]
    ssims = [0.974, 0.958, 0.955, 0.995, 0.998, 0.922]
    lpips = [0.078, 0.075, 0.041, 0.059, 0.004, 0.095]
    nmaes = [12.240, 29.484, 14.927, 1.548, 9.234, 42.870]
    train_times = [np.nan]*len(psnrs)
    model_sizes = [np.nan]*len(psnrs)
    scene_stats = []
    for p, s, l, n, tt, ms in zip(psnrs, ssims, lpips, nmaes, train_times, model_sizes):
      scene_stats.append({'psnr': p, 'ssim': s, 'lpips': l, 'normals_mae': n, 'num_hours': tt, 'mega_params': ms})
    avg_stats.append({k: type(scene_stats[0][k])(np.mean([s[k] for s in scene_stats])) for k in scene_stats[0].keys()})
    all_stats.append(scene_stats)
    continue;

  scene_stats = []
  for scene_name in scene_names:
    folder = os.path.join(root_folder, model_path, scene_name)
    stats = scrape_folder(folder, NUM_ITERS, metric_names=metric_names)
    print(model_path, scene_name, stats)
    scene_stats.append(stats)
  avg_stats.append({k: type(scene_stats[0][k])(np.mean([s[k] for s in scene_stats])) for k in scene_stats[0].keys()})
  all_stats.append(scene_stats)
  print(model_path, avg_stats[-1])

blender_refnerf_453115176 coffee {'psnr': 34.890018711090086, 'ssim': 0.9749827212095261, 'lpips': 0.07588588723912836, 'normals_mae': 11.845668504500003, 'num_hours': 6.575390185535848, 'mega_params': 0.71323}
blender_refnerf_453115176 helmet3 {'psnr': 29.714286947250365, 'ssim': 0.9552837216854095, 'lpips': 0.08357305970042944, 'normals_mae': 36.671281015, 'num_hours': 6.560205667893172, 'mega_params': 0.71323}
blender_refnerf_453115176 musclecar {'psnr': 31.08492564201355, 'ssim': 0.9564637777209282, 'lpips': 0.04047968026250601, 'normals_mae': 15.1815526225, 'num_hours': 6.561444666399807, 'mega_params': 0.71323}
blender_refnerf_453115176 phong_envmap {'psnr': 38.04869948387146, 'ssim': 0.9839657709002495, 'lpips': 0.0813480182737112, 'normals_mae': 4.32499015, 'num_hours': 6.565148490103001, 'mega_params': 0.71323}
blender_refnerf_453115176 teapot2 {'psnr': 45.91226264953613, 'ssim': 0.9964187118411064, 'lpips': 0.009179396553663536, 'normals_mae': 26.659667385, 'num_hours': 6.568

In [ ]:
names = list(models_meta.values())
data = np.stack([list(s.values()) for s in avg_stats])
precisions = [2, 3, 3, 2, 2, 1]
rank_order = [1, 1, -1, -1, 0, 0]  # +1 = higher is better, -1 = lower is better, 0 = do not color code
suffixes = ['', '', '', '', '', 'M']
render_table(names, data, precisions, rank_order, suffixes=suffixes)

ref-NeRF (from the paper) &    \cellcolor{red}35.96 &    \cellcolor{red}0.967 &    \cellcolor{red}0.059 &    \cellcolor{red}18.38 &  -  &  -  \\
ref-NeRF 360              & \cellcolor{orange}34.19 & \cellcolor{orange}0.964 & \cellcolor{orange}0.066 & \cellcolor{orange}22.96 & 6.57 & 0.7M \\


In [ ]:
names = list(models_meta.values())

precisions = [2, 3, 3, 2]
rank_orders = [1, 1, -1, -1]

name_map = {s: s for s in scene_names}
name_map['coffee'] = 'coffee'
name_map['teapot2'] = 'teapot'
name_map['musclecar'] = 'car'
name_map['phong_envmap'] = 'ball'
name_map['helmet3'] = 'helmet'
name_map['toaster'] = 'toaster'
pretty_scene_names = [name_map[s] for s in scene_names]

for i_metric, metric in enumerate(['psnr', 'ssim', 'lpips', 'normals_mae']):
  print(metric)
  precision = precisions[i_metric]
  rank_order = rank_orders[i_metric]

  print(' & ' + ' & '.join(['\\textit{' + s + '}' for s in pretty_scene_names]) + ' \\\\\\hline')
  data = np.array([np.array([s[metric] for s in scene_stats]) for scene_stats in all_stats])
  render_table(names, data, [precision] * len(scene_names), [rank_order] * len(scene_names))
  print()

psnr
 & \textit{coffee} & \textit{helmet} & \textit{car} & \textit{ball} & \textit{teapot} & \textit{toaster} \\\hline
ref-NeRF (from the paper) & \cellcolor{orange}34.21 & \cellcolor{orange}29.68 & \cellcolor{orange}30.82 &    \cellcolor{red}47.46 &    \cellcolor{red}47.90 &    \cellcolor{red}25.70 \\
ref-NeRF 360              &    \cellcolor{red}34.89 &    \cellcolor{red}29.71 &    \cellcolor{red}31.08 & \cellcolor{orange}38.05 & \cellcolor{orange}45.91 & \cellcolor{orange}25.51 \\

ssim
 & \textit{coffee} & \textit{helmet} & \textit{car} & \textit{ball} & \textit{teapot} & \textit{toaster} \\\hline
ref-NeRF (from the paper) & \cellcolor{orange}0.974 &    \cellcolor{red}0.958 & \cellcolor{orange}0.955 &    \cellcolor{red}0.995 &    \cellcolor{red}0.998 &    \cellcolor{red}0.922 \\
ref-NeRF 360              &    \cellcolor{red}0.975 & \cellcolor{orange}0.955 &    \cellcolor{red}0.956 & \cellcolor{orange}0.984 & \cellcolor{orange}0.996 & \cellcolor{orange}0.918 \\

lpips
 & \textit{cof